## Варіант 5
 ### Завдання  1

 Побудувати кубічний сплайн для таблично заданої функції, з граничними умовами для першої похідної.


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import display, Markdown, Math
from matplotlib.pyplot import figure
from numpy.polynomial import polynomial
from scipy.sparse import diags

def init_fig():
    global figure
    figure(num=None, figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;margin:0px auto;}
.tg td{border-style:solid;border-width:0px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;
  padding:10px 5px;word-break:normal;}
.tg th{border-style:solid;border-width:0px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<tbody>
  <tr>
    <td class="tg-c3ow">$i$</td>
    <td class="tg-c3ow">0</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">2</td>
    <td class="tg-c3ow">3</td>
    <td class="tg-0pky">$S'$</td>
  </tr>
  <tr>
    <td class="tg-c3ow">$X$</td>
    <td class="tg-c3ow">-4</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">3</td>
    <td class="tg-c3ow">7</td>
    <td class="tg-0pky">$S'(a)=-1$</td>
  </tr>
  <tr>
    <td class="tg-0lax">$Y$</td>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">5</td>
    <td class="tg-0lax">-3</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">$S'(b)=2$</td>
  </tr>
</tbody>
</table>

Побудова сплайну буде проводитись за першим способом з файлу, що був надісланий ("КУБІЧНА СПЛАЙН -НТЕРПОЛЯЦІЯ.pdf")

Сплайн матиме вигляд $S(x)=a_{i}+b_{i}(x-x_{i-1})+c_{i}(x-x_{i-1})^{2}+d_{i}(x-x_{i-1})^{3}$

В завданні задані граничні перші похідні, отже будуть використовуватися крайові умови першого типу в точках $x_0$ та $x_N$.

$$S^{\prime}\left(x_{0}\right)=f^{\prime}\left(x_{0}\right) ; \quad S^{\prime}\left(x_{N}\right)=f^{\prime}\left(x_{N}\right)$$

$$\left\{\begin{array}{l}b_{1}=f^{\prime}\left(x_{0}\right) \\ b_{N}+2 c_{N} h_{N}+3 d_{N} h_{n}^{2}=f^{\prime}\left(x_{N}\right)\end{array}\right.$$

Для знаходження коефіцієнтів $c_i$ треба знайти розв'язок наступної СЛАР

$$
\left\{\begin{array}{l}
2 c_{1}+c_{2}=3 \dfrac{y_{1}-y_{0}}{h_{1}^{2}}-\dfrac{3}{h_{1}} f^{\prime} x_{0} \\
h_{i-1} c_{i-1}+2 c_{i}\left(h_{i}+h_{i-1}\right)+h_{i} c_{i+1}=3\left[\dfrac{y_{i}-y_{i-1}}{h_{i}}-\dfrac{y_{i-1}-y_{i-2}}{h_{i-1}}\right], \quad i=\overline{2, N-1} \\
\dfrac{2}{3} h_{N-1} c_{N-1}+\left(h_{N}+\dfrac{4}{3} h_{N-1}\right) c_{N}=-f^{\prime}\left(x_{N}\right)-2 \dfrac{y_{N-1}-y_{N-2}}{h_{N-1}}+3 \dfrac{y_{N}-y_{N-1}}{h_{N}}
\end{array}\right.
$$

$h_i$ - різниці $x_i - x_{i-1}, i=\overline{1, N}$

In [143]:
X = np.array(
#     [-4, 1, 3, 7]
    [-3, 2, 5, 6]
)
Y = np.array(
#     [2, 5, -3, 0]
    [-2, 1, 3, 4]
)
# Spr_a = -1
Spr_a = -3
# Spr_b = 2
Spr_b = 1

H = X[1:] - X[:-1]

# Warning
# index N in formulas is [-1] in arrays 

#################################################      DISPLAY
display(Markdown("#### Таблиця вхідних точок"))
table = pd.concat([
        pd.DataFrame(X),
        pd.DataFrame(Y),
        pd.DataFrame(np.concatenate([[0], H])),
    ], axis=1)
table.columns = [
    '$x_i$',
    '$y_i$',
    '$h_i$',
    ]
display(table.T)
###############################################################

# diagonal-based input
C_coef_unsolved = diags(
    [
        np.concatenate((H[:-2], np.array([2/3*H[-2]]) )),
        np.concatenate((np.array([2]), (H[1:-1]+H[0:-2])*2, np.array([H[-1]+4/3*H[-2]]) )), 
        np.concatenate((np.array([1]), H[1:-1]))
    ],
    [-1, 0, 1],
    shape=[len(H)]*2
).toarray()
# equals b
b = np.concatenate((
    np.array([ 3*(Y[1]-Y[0])/H[0]**2 - 3/H[0]*Spr_a ]),
    3*((Y[2:-1]-Y[1:-2])/H[1:-1] - (Y[1:-2]-Y[0:-3])/H[0:-2]),
    np.array([ -Spr_b - 2*((Y[-2] - Y[-3])/H[-2]) + 3*( (Y[-1] - Y[-2])/H[-1] ) ])
))
# solve the system of linear equations
C = np.linalg.solve(C_coef_unsolved, b)

#################################################      DISPLAY
display(Markdown("#### Трьохдіагональна матриця системи"))
display(pd.DataFrame(C_coef_unsolved))
display(Markdown("#### Вектор-стовпчик вільних членів"))
display(pd.DataFrame(b))
display(Markdown("#### Значення коефіцієнтів $c_i$"))
display(pd.DataFrame(C, columns=["$c_i$"], index=np.arange(1,C.size+1)))
###############################################################


#### Таблиця вхідних точок

,0,1,2,3
$x_i$,-3,2,5,6
$y_i$,-2,1,3,4
$h_i$,0,5,3,1


#### Трьохдіагональна матриця системи

,0,1,2
0,2.0,1.0,0.0
1,5.0,16.0,3.0
2,0.0,2.0,5.0


#### Вектор-стовпчик вільних членів

,0
0,2.160000
1,0.200000
2,0.666667


#### Значення коефіцієнтів $c_i$

,$c_i$
1,1.307642
2,-0.455285
3,0.315447


Знаходження решти коефіцієнтів буде відбуватись згідно системи нижче
$$
\left\{\begin{array}{l}
a_{i}=y_{i-1}, \quad i=\overline{1, N}\\
b_{i}=\dfrac{y_{i}-y_{i-1}}{h_{i}}-\dfrac{h_{i}}{3}\left(c_{i+1}+2 c_{i}\right), \quad i=\overline{1, N-1} \\
b_{N}=\dfrac{y_{N-1}-y_{N-2}}{h_{N-1}}+\dfrac{h_{N-1}}{3}\left(2 c_{N}+c_{N-1}\right) \\
d_{i}=\dfrac{c_{i+1}-c_{i}}{3 h_{i}}, \quad i=\overline{1, N-1} \\
d_{N}=\dfrac{f^{\prime}\left(x_{n}\right)-b_{N}-2 c_{N} h_{N}}{3 h_{N}^{2}} \\
\end{array}\right.
$$

In [185]:
A = Y[:-1]

B_i = (Y[1:-1] - Y[0:-2])/H[:-1] - H[:-1]/3*(C[1:] + 2*C[:-1])
B_N = np.array([(Y[-2] - Y[-3])/H[-2] + H[-2]/3*(2*C[-1] + C[-2])])
B = np.concatenate([B_i, B_N])

D_i = (C[1:] - C[:-1])/(3*H[:-1])
D_N = np.array([( Spr_b - B[-1] - 2*C[-1]*H[-1] )/(3*H[-1]**2)])
D = np.concatenate(( D_i, D_N ))

S = np.concatenate((
        A[:, np.newaxis],
        B[:, np.newaxis],
        C[:, np.newaxis],
        D[:, np.newaxis],
    ),
    axis=1
)
#################################################      DISPLAY
for i in range(len(X)-1):
    display(Markdown(f"$x \in \\left[{X[i]};{X[i+1]}\\right]$"))
    display(pd.DataFrame(
        S[i,:][np.newaxis, :],
        columns=[f'$a_{i+1}$',f'$b_{i+1}$',f'$c_{i+1}$',f'$d_{i+1}$'],
        index=[i+1]
    ).T)
###############################################################


$x \in \left[-3;2\right]$

,1
$a_1$,-2.000000
$b_1$,-3.000000
$c_1$,1.307642
$d_1$,-0.117528


$x \in \left[2;5\right]$

,2
$a_2$,1.000000
$b_2$,1.261789
$c_2$,-0.455285
$d_2$,0.085637


$x \in \left[5;6\right]$

,3
$a_3$,3.000000
$b_3$,0.842276
$c_3$,0.315447
$d_3$,-0.157724
